## Imports

In [50]:
import pandas as pd
import pickle
import spacy

## Loading Data

In [51]:
ekg_denoised = pd.read_pickle('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised.pkl')
ekg_denoised = ekg_denoised.sample(frac=1)


In [52]:
ekg_denoised.head()

,ICD9_CODE,TEXT
4034,2,probable ectopic atrial rhythm left axis devia...
606,0,sinus tachycardia prominent inferior waves - a...
811,0,sinus rhythm. left atrial abnormality. left ve...
5108,3,baseline artifact. rapid irregular rhythm. con...
6533,3,baseline artifact in lead vi. sinus rhythm. rs...


## Spacy

#### Downloading and Loading Spacy Embeddings

In [1]:
#!python -m spacy download en_core_web_md

2021-11-17 00:38:48.868995: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-17 00:38:48.869188: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     |████████████████████████████████| 45.7 MB 64 kB/s             
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [53]:
nlp = spacy.load('en_core_web_md')

#### Testing Spacy Embeddings

In [54]:
doc = nlp("sinus rhythm. compared to tracing 1 no diagnostic interim change. tracing 2")
print(doc.vector.shape)
doc.vector

(300,)


array([-5.00825867e-02,  2.34442070e-01, -1.47236139e-01,  1.79825034e-02,
       -3.41885015e-02, -5.65572195e-02,  9.41328239e-03, -4.99943532e-02,
        1.39321953e-01,  1.65153766e+00, -1.48617715e-01,  2.31521294e-01,
        8.35297704e-02,  1.76000714e-01,  1.16957293e-03, -5.54402284e-02,
        1.99811742e-01,  1.40888703e+00, -9.33964252e-02, -5.83115034e-02,
        3.43884304e-02,  3.40443780e-03, -2.34556273e-01, -1.05062291e-01,
       -8.92671105e-03,  1.92777123e-02, -8.44067335e-03,  1.91751588e-02,
        1.34894580e-01,  8.49958584e-02,  4.53957133e-02,  7.32796043e-02,
       -1.18446406e-02, -1.36181086e-01,  7.27292076e-02,  1.67326778e-01,
        1.02550071e-02,  8.56276304e-02,  6.58611134e-02, -6.31382093e-02,
        2.15512812e-02, -7.76198581e-02,  7.92574957e-02, -2.48679277e-02,
       -3.74911427e-02, -1.35982186e-01, -9.75067765e-02,  1.94221720e-01,
       -1.74879972e-02,  3.41023579e-02, -1.35499965e-02, -3.56617868e-02,
        9.32188630e-02, -

#### Running Spacy on dataset, saving vectors off

In [55]:
docs = list(nlp.pipe(ekg_denoised.TEXT))

In [56]:
spacy_embeddings = []
for i in docs:
    spacy_embeddings.append(i.vector)

In [57]:
ekg_denoised['spacy_embeddings'] = spacy_embeddings
ekg_denoised.drop(columns='TEXT'
, inplace = True)

In [59]:
ekg_denoised.to_pickle('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised_spacy.pkl')


In [1]:
# Averaging could be losing a lot of information